In [13]:
!pip install yfinance

In [24]:
# IMPORTS
import numpy as np
import pandas as pd
import requests
import re


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

### Question 1: [IPO] Withdrawn IPOs by Company Type

What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?



From the withdrawn IPO list (stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

In [43]:
import pandas as pd
import requests
from io import StringIO

def get_withdrawn_ipos(url: str) -> pd.DataFrame:
    """
    Fetch and return HTML tables from the given URL using pandas.
    Returns the first table if multiple are found.
    """
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found on the page {url}.")

        # Return all tables or only the first depending on your needs
        return tables[0]  # or return tables to get all

    except requests.exceptions.RequestException as e:
        print(f"Request failed for {url}: {e}")
    except ValueError as ve:
        print(f"Data error for {url}: {ve}")
    except Exception as ex:
        print(f"An unexpected error occurred for {url}: {ex}")

    return pd.DataFrame()

In [59]:
withdrawn_ipos_df = get_withdrawn_ipos("https://stockanalysis.com/ipos/withdrawn/")

In [46]:
len(withdrawn_ipos_df)

99

In [47]:
withdrawn_ipos_df.head()

,Symbol,Company Name,Price Range,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
1,AURN,"Aurion Biotech, Inc.",-,-
2,ROTR,"PHI Group, Inc.",-,-
3,ONE,One Power Company,-,-
4,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000


In [48]:
withdrawn_ipos_df.columns

Index(['Symbol', 'Company Name', 'Price Range', 'Shares Offered'], dtype='object')

Create a new column called Company Class, categorizing company names based on patterns

In [49]:
def categorize_company_class(df: pd.DataFrame) -> pd.DataFrame:
    """
    Categorizes company names into 'Company Class' based on predefined patterns.

    The function converts company names to lowercase and splits them into words
    for robust pattern matching.

    Args:
        df (pd.DataFrame): The input DataFrame containing a 'Company Name' column.

    Returns:
        pd.DataFrame: The DataFrame with a new 'Company Class' column.
                      Returns an empty DataFrame if 'Company Name' column is missing.
    """
    if 'Company Name' not in df.columns:
        print("Error: 'Company Name' column not found in the DataFrame.")
        return pd.DataFrame()

    # Define the patterns and their corresponding categories
    # The order matters: more specific or dominant categories should be checked first
    # However, in this simplified set, the order for distinct keywords is less critical.
    patterns = {
        'corp': ['corp', 'corporation'], # Any company with 'corp' or 'corporation'
        'inc': ['inc', 'incorporated'],
        'group': ['group'],
        'holdings': ['holdings'],
        'ltd': ['ltd', 'limited']
    }

    def get_company_class(company_name: str) -> str:
        """Helper function to determine the company class for a single name."""
        if not isinstance(company_name, str):
            return "Other"

        # Convert to lowercase for case-insensitive matching
        lower_name = company_name.lower()

        # Iterate through patterns and apply the first match found
        for category, keywords in patterns.items():
            for keyword_phrase in keywords:
                # Check if the entire keyword phrase is present in the lowercased company name
                if keyword_phrase in lower_name:
                    return category
        return "Other" # Default category if no pattern matches

    # Apply the categorization function to the 'Company Name' column
    df['Company Class'] = df['Company Name'].apply(get_company_class)

    return df

In [50]:
categorize_company_class(withdrawn_ipos_df)

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,inc
1,AURN,"Aurion Biotech, Inc.",-,-,inc
2,ROTR,"PHI Group, Inc.",-,-,inc
3,ONE,One Power Company,-,-,Other
4,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000,holdings
...,...,...,...,...,...
94,FHP,"Freehold Properties, Inc.",-,-,inc
95,CHO,Chobani Inc.,-,-,inc
96,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,inc
97,GLGX,"Gerson Lehrman Group, Inc.",-,-,inc


In [51]:
withdrawn_ipos_df.groupby('Company Class').size()

,0
Company Class,
Other,4
corp,24
group,4
holdings,5
inc,50
ltd,12


Define a new field Avg. price by parsing the Price Range field (create a function and apply it to the Price Range column).


In [52]:
def calculate_average_price(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the 'Avg. Price' from the 'Price Range' column in a DataFrame.

    Handles price ranges (e.g., '$8.00-$10.00'), single prices (e.g., '$5.00'),
    and non-available data (e.g., '-').

    Args:
        df (pd.DataFrame): The input DataFrame containing a 'Price Range' column.

    Returns:
        pd.DataFrame: The DataFrame with a new 'Avg. Price' column.
                      Returns an empty DataFrame if 'Price Range' column is missing.
    """
    if 'Price Range' not in df.columns:
        print("Error: 'Price Range' column not found in the DataFrame.")
        return pd.DataFrame()

    def parse_price_range(price_range_str: str):
        """Helper function to parse a single price range string."""
        if not isinstance(price_range_str, str):
            return None

        # Remove '$' and any leading/trailing whitespace
        cleaned_str = price_range_str.replace('$', '').strip()

        if cleaned_str == '-':
            return None
        elif '-' in cleaned_str:
            try:
                # Split the range, convert parts to float, and calculate average
                lower_price, upper_price = map(float, cleaned_str.split('-'))
                return (lower_price + upper_price) / 2.0
            except ValueError:
                # Handle cases where conversion to float fails
                print(f"Warning: Could not parse price range '{price_range_str}'. Returning None.")
                return None
        else:
            try:
                # Single price value
                return float(cleaned_str)
            except ValueError:
                # Handle cases where conversion to float fails
                print(f"Warning: Could not parse single price '{price_range_str}'. Returning None.")
                return None

    # Apply the parsing function to the 'Price Range' column
    df['Avg. Price'] = df['Price Range'].apply(parse_price_range)

    return df

In [53]:
calculate_average_price(withdrawn_ipos_df)

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,inc,3.5
1,AURN,"Aurion Biotech, Inc.",-,-,inc,NaN
2,ROTR,"PHI Group, Inc.",-,-,inc,NaN
3,ONE,One Power Company,-,-,Other,NaN
4,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000,holdings,5.0
...,...,...,...,...,...,...
94,FHP,"Freehold Properties, Inc.",-,-,inc,NaN
95,CHO,Chobani Inc.,-,-,inc,NaN
96,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,inc,19.5
97,GLGX,"Gerson Lehrman Group, Inc.",-,-,inc,NaN


Convert Shares Offered to numeric, clean missing or invalid values.

In [54]:
withdrawn_ipos_df['Shares Offered'] = pd.to_numeric(withdrawn_ipos_df['Shares Offered'], errors='coerce')

Create a new column:
Withdrawn Value = Shares Offered * Avg Price

In [55]:
withdrawn_ipos_df['Withdrawn Value'] = withdrawn_ipos_df['Shares Offered'] * withdrawn_ipos_df['Avg. Price']

In [56]:
withdrawn_ipos_df['Withdrawn Value'].notna().sum()

np.int64(71)

Group by Company Class and calculate total withdrawn value.

In [58]:
withdrawn_ipos_df.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False)

,Withdrawn Value
Company Class,
corp,4.111850e+09
inc,2.257164e+09
Other,7.520700e+08
ltd,3.217346e+08
holdings,2.280000e+08
group,3.378750e+07


Which class had the highest total value of withdrawals?

The Class Corp had the highest total value of withdrawals eventhough the class inc had the greatest number of IPOs.

### Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)

What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

In [81]:
withdrawn_2024_df = get_withdrawn_ipos("https://stockanalysis.com/ipos/2024/")

In [82]:
withdrawn_2024_df.columns

Index(['IPO Date', 'Symbol', 'Company Name', 'IPO Price', 'Current', 'Return'], dtype='object')

In [83]:
# Ensure 'IPO Date' is datetime
withdrawn_2024_df['IPO Date'] = pd.to_datetime(withdrawn_2024_df['IPO Date'], errors='coerce')

# Drop rows with any missing values
withdrawn_2024_df.replace('-', np.nan, inplace=True)
withdrawn_2024_df = withdrawn_2024_df.dropna()

# Filter for IPOs before 1 June 2024
filtered_df = withdrawn_2024_df[withdrawn_2024_df['IPO Date'] < pd.Timestamp("2024-06-01")]

# Optional: Reset index if needed
filtered_df = filtered_df.reset_index(drop=True)

In [85]:
filtered_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$35.43,108.94%
1,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$20.41,4.35%
2,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.50,5.00%
3,2024-05-15,JDZG,JIADE Limited,$4.00,$0.28,-92.63%
4,2024-05-15,RAY,Raytech Holding Limited,$4.00,$1.21,-68.75%
...,...,...,...,...,...,...
70,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$1.08,-81.83%
71,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$13.92,-15.56%
72,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$1.79,-58.00%
73,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$20.19,-7.76%


Download daily stock data for those tickers (via yfinance).
Make sure you understand how growth_1d ... growth_365d, and volatility columns are defined.Define a new column growth_252d representing growth after 252 trading days (~1 year), in addition to any other growth periods you already track.

In [111]:
import time

def get_stock_features(ALL_TICKERS):
    """
    Fetches historical stock data for a list of tickers and generates specified features,
    including a 'Ticker' column for identification.

    Args:
        ALL_TICKERS (list): A list of stock ticker symbols.

    Returns:
        pd.DataFrame: A DataFrame containing historical stock data with engineered features
                      and a 'Ticker' column.
    """
    stocks_df = pd.DataFrame() # Initialize an empty DataFrame

    for i, ticker in enumerate(ALL_TICKERS):
        #print(i, ticker)

        ticker_obj = yf.Ticker(ticker)
        historyPrices = ticker_obj.history(period="max", interval="1d")

        # Add the Ticker column back
        historyPrices['Ticker'] = ticker

        # historical returns
        for j in [1, 3, 7, 30, 90, 252, 365]:
            historyPrices['growth_' + str(j) + 'd'] = historyPrices['Close'] / historyPrices['Close'].shift(j)

        # volataility
        historyPrices['volatility'] = historyPrices['Close'].rolling(30).std() * np.sqrt(252)

        # sleep 1 sec between downloads - not to overload the API server
        time.sleep(1)

        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=False)

    return stocks_df

In [112]:
full_stock_data_df = get_stock_features(filtered_df['Symbol'].unique().tolist())

In [115]:
full_stock_data_df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d,volatility,Sharpe
Date,,,,,,,,,,,,,,,,,
2025-06-03 00:00:00-04:00,3.36,3.5300,3.160,3.41,67200,0.0,0.0,ROMA,1.014881,1.100000,1.713568,3.195876,4.552737,5.262346,NaN,10.986046,0.474907
2025-06-04 00:00:00-04:00,3.50,3.8500,3.457,3.76,271100,0.0,0.0,ROMA,1.102639,1.220779,1.634072,3.387387,5.115646,5.829458,NaN,11.789832,0.490631
2025-06-05 00:00:00-04:00,3.74,4.1350,3.360,3.57,264600,0.0,0.0,ROMA,0.949468,1.062500,1.275000,3.230769,4.760000,6.144579,NaN,12.173744,0.501044
2025-06-06 00:00:00-04:00,3.65,3.9500,3.630,3.70,84400,0.0,0.0,ROMA,1.036415,1.085044,1.237458,3.262787,5.211268,6.156406,NaN,12.553202,0.486840
2025-06-09 00:00:00-04:00,3.75,3.9547,2.612,2.86,381644,0.0,0.0,ROMA,0.772973,0.760638,0.922581,2.454936,4.386503,4.627832,NaN,12.238182,0.374470


Calculate the Sharpe ratio assuming a risk-free rate of 4.5%

In [114]:
full_stock_data_df['Sharpe'] = (full_stock_data_df['growth_252d'] - 0.045) / full_stock_data_df['volatility']

In [116]:
if not isinstance(full_stock_data_df.index, pd.DatetimeIndex):
    full_stock_data_df.index = pd.to_datetime(full_stock_data_df.index)

filtered_data_20250606 = full_stock_data_df.loc['2025-06-06']

In [117]:
filtered_data_20250606[['growth_252d', 'Sharpe']].describe()

,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152898,0.301597
std,1.406017,0.529685
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.335681
max,8.097413,2.835668


    What is the median Sharpe ratio for these 71 stocks?

Based on the describe() output:

    The median (50%) Sharpe ratio for these 71 stocks is 0.083768.


[Additional] Do you observe the same top 10 companies when sorting by growth_252d versus sorting by Sharpe?


In [118]:
# Get top 10 companies by growth_252d
top_10_by_growth = filtered_data_20250606.sort_values(by='growth_252d', ascending=False)['Ticker'].head(10).tolist()
print(f"Top 10 companies by growth_252d:\n{top_10_by_growth}\n")

Top 10 companies by growth_252d:
['JL', 'ROMA', 'UMAC', 'NNE', 'RBRK', 'AHR', 'AS', 'MRX', 'RDDT', 'MTEN']



In [119]:
# Get top 10 companies by Sharpe ratio
top_10_by_sharpe = filtered_data_20250606.sort_values(by='Sharpe', ascending=False)['Ticker'].head(10).tolist()
print(f"Top 10 companies by Sharpe ratio:\n{top_10_by_sharpe}\n")

Top 10 companies by Sharpe ratio:
['BKHA', 'JVSA', 'LEGT', 'IBAC', 'NCI', 'HLXB', 'MNDR', 'DYCQ', 'INTJ', 'JL']



In [121]:
# Compare the two lists
common_companies = set(top_10_by_growth).intersection(set(top_10_by_sharpe))
print(f"There are {len(common_companies)} common companies in the top 10 lists:")
print(common_companies)

There are 1 common companies in the top 10 lists:
{'JL'}


### Question 3: [IPO] ‘Fixed Months Holding Strategy’
What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?
(Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)

In [124]:
def get_stock_data_with_future_growth(ALL_TICKERS):
    """
    Fetches historical stock data for a list of tickers and generates specified features,
    including past growth, volatility, ticker identification, and future growth over 1-12 months.

    Args:
        ALL_TICKERS (list): A list of stock ticker symbols.

    Returns:
        pd.DataFrame: A DataFrame containing historical stock data with engineered features,
                      including future growth columns, and a DatetimeIndex for date tracking.
    """
    stocks_df = pd.DataFrame() # Initialize an empty DataFrame

    # Define the number of trading days for future growth calculations (1 to 12 months)
    # Assuming 1 month = 21 trading days
    future_growth_days = [i * 21 for i in range(1, 13)] # [21, 42, 63, ..., 252]

    for i, ticker in enumerate(ALL_TICKERS):
        # print(f"{i+1}/{len(ALL_TICKERS)}: Downloading {ticker} data...")

        ticker_obj = yf.Ticker(ticker)
        historyPrices = ticker_obj.history(period="max", interval="1d")

        if historyPrices.empty:
            print(f"Warning: No historical data found for {ticker}. Skipping.")
            time.sleep(1) # Still sleep to avoid hammering API
            continue

        # Add the Ticker column
        historyPrices['Ticker'] = ticker

        # NEW: Add future growth columns (1 to 12 months)
        for m, days_shifted in enumerate(future_growth_days):
            col_name = f'future_growth_{m+1}m'
            # Calculate future growth: Close price 'days_shifted' days in the future divided by current Close
            historyPrices[col_name] = historyPrices['Close'].shift(-days_shifted) / historyPrices['Close']

        # Sleep 1 sec between downloads - not to overload the API server
        time.sleep(1)

        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            # Using ignore_index=False to preserve the DatetimeIndex for filtering by date
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=False)

    return stocks_df

In [125]:
ipo_full_stock_data_df = get_stock_data_with_future_growth(filtered_df['Symbol'].unique().tolist())

In [126]:
print(ipo_full_stock_data_df.head())
print(ipo_full_stock_data_df.columns)

                                Open       High        Low      Close  \
Date                                                                    
2024-05-23 00:00:00-04:00  23.000000  24.270000  22.139999  23.799999   
2024-05-24 00:00:00-04:00  24.260000  26.150000  23.980000  25.700001   
2024-05-28 00:00:00-04:00  25.850000  26.879999  25.075001  26.480000   
2024-05-29 00:00:00-04:00  26.440001  26.490000  25.500999  26.290001   
2024-05-30 00:00:00-04:00  27.209999  27.209999  25.500000  26.139999   

                            Volume  Dividends  Stock Splits Ticker  growth_1d  \
Date                                                                            
2024-05-23 00:00:00-04:00  3335800        0.0           0.0    BOW        NaN   
2024-05-24 00:00:00-04:00   990500        0.0           0.0    BOW   1.079832   
2024-05-28 00:00:00-04:00   555100        0.0           0.0    BOW   1.030350   
2024-05-29 00:00:00-04:00   302700        0.0           0.0    BOW   0.992825   
20

Determine the first trading day (min_date) for each ticker.
This is the earliest date in the data for each stock.

In [128]:
if not isinstance(ipo_full_stock_data_df.index, pd.DatetimeIndex):
    ipo_full_stock_data_df.index = pd.to_datetime(ipo_full_stock_data_df.index)

# Group by 'Ticker' and find the minimum date (from the index) for each
first_trading_days = ipo_full_stock_data_df.groupby('Ticker').apply(lambda x: x.index.min(), include_groups=False)
print("First Trading Day for each Ticker:")
print(first_trading_days.head()) # Print first few entries to verify
print(f"\nTotal tickers: {len(first_trading_days)}")

First Trading Day for each Ticker:
Ticker
AHR    2024-02-07 00:00:00-05:00
ALAB   2024-03-20 00:00:00-04:00
ANRO   2024-02-02 00:00:00-05:00
AS     2024-02-01 00:00:00-05:00
AUNA   2024-03-22 00:00:00-04:00
dtype: datetime64[ns, America/New_York]

Total tickers: 75


Join the data:
Perform an inner join between the min_date DataFrame and the future growth data on both ticker and date.

In [129]:
# 1. Prepare ipo_full_stock_data_df: Reset index to make 'Date' a column
ipo_full_stock_data_df_reset = ipo_full_stock_data_df.reset_index()
ipo_full_stock_data_df_reset.rename(columns={'index': 'Date'}, inplace=True)

In [130]:
# 2. Prepare first_trading_days: Convert Series to DataFrame and rename columns
first_trading_days_df = first_trading_days.reset_index()
first_trading_days_df.columns = ['Ticker', 'IPO_Date']

In [132]:
# Ensure 'Date' and 'IPO_Date' columns are of datetime type for accurate joining
ipo_full_stock_data_df_reset['Date'] = pd.to_datetime(ipo_full_stock_data_df_reset['Date'])
first_trading_days_df['IPO_Date'] = pd.to_datetime(first_trading_days_df['IPO_Date'])

In [136]:
merged_data = pd.merge(
    ipo_full_stock_data_df_reset,
    first_trading_days_df,
    how='inner',
    left_on=['Ticker', 'Date'],
    right_on=['Ticker', 'IPO_Date']
)

merged_data.drop(columns=['IPO_Date'], inplace=True)

In [137]:
print(f"\nShape of the resulting DataFrame: {merged_data.shape}")


Shape of the resulting DataFrame: (75, 29)




Compute descriptive statistics for the resulting DataFrame:
Use .describe() or similar to analyze each of the 12 columns:

    future_growth_1m
    future_growth_2m
    ...
    future_growth_12m



In [141]:
future_growth_columns = [f'future_growth_{i}m' for i in range(1, 13)]
merged_data[future_growth_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
future_growth_1m,75.0,0.927259,0.346261,0.098947,0.778984,0.977000,1.046509,2.646505
future_growth_2m,75.0,0.940544,0.574545,0.073800,0.685815,1.000000,1.154013,4.874759
future_growth_3m,75.0,0.833825,0.409763,0.060947,0.511212,0.927500,1.069085,2.040000
future_growth_4m,75.0,0.825087,0.401969,0.045368,0.517233,0.909091,1.134300,1.605000
future_growth_5m,75.0,0.803769,0.488349,0.054109,0.448403,0.821092,1.016381,3.213873
future_growth_6m,75.0,0.864185,0.653180,0.061432,0.384560,0.802239,1.093948,3.670520
future_growth_7m,75.0,0.847149,0.712943,0.044086,0.296870,0.844875,1.114468,5.122350
future_growth_8m,75.0,0.832984,0.762423,0.043103,0.208677,0.812109,1.082438,5.171484
future_growth_9m,74.0,0.892996,0.938269,0.033144,0.244048,0.833357,1.055767,6.764933
future_growth_10m,74.0,0.917945,0.911431,0.037769,0.242424,0.772592,1.200799,5.352601


Determine the best holding period:

    Find the number of months (1 to 12) where the average (mean) future growth is maximal.

From the above table we see that holding for 2 months has the highest avg growth . This still howeever is almost a 6 percent loss on the initial investment

### Question 4: [Strategy] Simple RSI-Based Trading Strategy

What is the total profit (in thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?